### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import shutil
import random

### Mounting drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Preparing Dataset

In [3]:
sr=r"/content/drive/MyDrive/brain_tumor_dataset/"
root_dir = r'/content/drive/MyDrive/brain_tumor_dataset/'
classes_dir = ['yes','no']

test_ratio = 0.20
#create folder train and test , every folder have two subfolder -->(yes and no)
for cls in classes_dir:
    os.makedirs(root_dir +'train/' + cls)
    os.makedirs(root_dir +'test/' + cls)

import tqdm
def call2(filen):#yes
    #define the src folder
    src = os.path.join(sr,filen)
    #read the src folder
    allFileNames = os.listdir(src)
    #randomize the files names
    np.random.shuffle(allFileNames)
    #split the files in train and test
    train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - test_ratio))])

    #add the full url
    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print("*****************************")
    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Testing: ', len(test_FileNames))
    print("*****************************")

    #copy the train and test files in corresponding folder
    for name in tqdm.tqdm(train_FileNames):
        try:
            shutil.copy(name, root_dir +'train/'+filen+'/' )
        except:
            pass

    for name in tqdm.tqdm(test_FileNames):
        #if any error it ignores this file.
        try:
            shutil.copy(name, root_dir +'test/'+filen+'/')
        except:
            pass
    print("Copying Done!")

In [4]:
for x in classes_dir:
  # we called every class folder from here. --> accident ,noaccident
  call2(x)

*****************************
Total images:  155
Training:  124
Testing:  31
*****************************


100%|██████████| 31/31 [00:00<00:00, 42.05it/s]


Copying Done!
*****************************
Total images:  98
Training:  78
Testing:  20
*****************************


100%|██████████| 20/20 [00:00<00:00, 82.97it/s]

Copying Done!


### Model

In [28]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(180, 180, 3),
    pooling='max',
    classes=2,
)
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(1, activation='sigmoid'))

In [26]:
#resnet_model.summary()

In [29]:
resnet_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  sr,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  sr,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 506 files belonging to 4 classes.
Using 405 files for training.
Found 506 files belonging to 4 classes.
Using 101 files for validation.


In [31]:
epochs=10
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
13/13 [==============================] - 92s 7s/step - loss: -449.9695 - accuracy: 0.1012 - val_loss: -991.4415 - val_accuracy: 0.1089
Epoch 2/10
13/13 [==============================] - 83s 6s/step - loss: -1756.9133 - accuracy: 0.0988 - val_loss: -2466.3789 - val_accuracy: 0.1089
Epoch 3/10
13/13 [==============================] - 85s 6s/step - loss: -3583.4629 - accuracy: 0.0988 - val_loss: -4558.7212 - val_accuracy: 0.1089
Epoch 4/10
13/13 [==============================] - 82s 6s/step - loss: -6202.0029 - accuracy: 0.0988 - val_loss: -7378.2969 - val_accuracy: 0.1089
Epoch 5/10
13/13 [==============================] - 90s 7s/step - loss: -9559.4756 - accuracy: 0.0988 - val_loss: -11000.0732 - val_accuracy: 0.1089
Epoch 6/10
13/13 [==============================] - 85s 6s/step - loss: -13748.3936 - accuracy: 0.0988 - val_loss: -15537.8086 - val_accuracy: 0.1089
Epoch 7/10
13/13 [==============================] - 85s 7s/step - loss: -19160.7305 - accuracy: 0.0988 - val_lo